In [1]:
import pandas as pd
# from tabulate import tabulate
import numpy as np
from pygobnilp.gobnilp import Gobnilp
# import bnlearn as bn
import pgmpy
import itertools
from scipy.stats import chi2_contingency #check for independence between 2 variables
from scipy.stats import power_divergence
from pgmpy.estimators.CITests import log_likelihood
from pgmpy.estimators.CITests import chi_square
# from pgmpy.factors.continuous.discretize import BaseDiscretizer

In [2]:
def csv_format_discrete(csv_file):
    df = pd.read_csv(csv_file, sep="\s+")
    return df.drop([0])

#returns the csv_file in a pandas dataframe, formatted properly, discrete dataset only

In [3]:
df = csv_format_discrete(r"C:\Users\User\Documents\GitHub\ML_FYP\dataset\discrete.dat")
df
df_test = csv_format_discrete(r"C:\Users\User\Documents\GitHub\ML_FYP\dataset\alarm_10000.dat")
df_test

,PULMEMBOLUS,PAP,KINKEDTUBE,INTUBATION,MINVOLSET,VENTMACH,DISCONNECT,VENTTUBE,VENTLUNG,SHUNT,...,CO,BP,EXPCO2,ERRLOWOUTPUT,PRESS,HRBP,MINVOL,HISTORY,HRSAT,PCWP
1,1,1,0,1,2,2,1,3,3,0,...,1,1,1,0,0,0,3,0,0,2
2,0,2,0,1,2,2,0,1,3,1,...,0,0,1,1,1,2,3,0,0,2
3,0,2,0,2,2,2,0,1,1,0,...,0,0,2,0,0,0,1,0,0,2
4,0,2,0,1,2,2,0,1,3,1,...,1,1,1,0,0,0,3,1,2,1
5,0,2,0,1,2,2,0,1,3,1,...,0,0,1,0,1,1,3,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,0,2,1,1,2,2,0,1,3,1,...,1,1,1,0,1,0,3,0,0,0
9997,0,0,0,1,2,2,0,1,3,1,...,0,0,1,0,2,0,3,0,0,2
9998,0,2,0,0,0,0,0,0,2,1,...,0,0,2,0,0,0,3,0,0,2
9999,0,2,0,1,2,2,0,1,3,1,...,1,1,1,0,1,0,3,0,0,0


### chi-squared test

In [4]:
#0th order chi2 test
def chi2bool(df):
#returns a tuple(chi2, p_value, dof) if boolean = false
#the null hypothesis is that they are independent of each other
#if true, the p_value is higher than the significance test, we do not reject the null hypothesis
#if false, the p_value is lower than the significance test, we reject the null hypothesis
    v = list(df)
    empty = []
    empty_1 = []
    empty_2 = []
    empty_3 = []
    empty_4 = []
    itertools_combinations = list(itertools.combinations(v, 2)) #finds every possible combination of list(df)
    for i in itertools_combinations:
        empty_1.append(i[0])
        empty_2.append(i[1])
        
    total_number_of_combinations = len(empty_1)
    
    for t in range(len(itertools_combinations)):
        empty.append(itertools_combinations[t])
        
    for value in range(len(empty_1)):
        empty_3.append(value)
        
    for x,y,i,j in zip(empty_1, empty_2, empty, empty_3):
        chisquare = chi_square(X=x, Y=y, Z=[], data=df, significance_level=0.05, boolean=True) #returns chi, p_value, dof
        empty_4.append([j, i , chisquare])
        
    return empty_4

In [5]:
def chi2val(df):
#returns a tuple(chi2, p_value, dof) if boolean = false
#the null hypothesis is that they are independent of each other
#if true, the p_value is higher than the significance test, we do not reject the null hypothesis
#if false, the p_value is lower than the significance test, we reject the null hypothesis
    v = list(df)
    empty = []
    empty_1 = []
    empty_2 = []
    empty_3 = []
    empty_4 = []
    itertools_combinations = list(itertools.combinations(v, 2)) #finds every possible combination of list(df)
    for i in itertools_combinations:
        empty_1.append(i[0])
        empty_2.append(i[1])
        
    total_number_of_combinations = len(empty_1)
    
    for t in range(len(itertools_combinations)):
        empty.append(itertools_combinations[t])
        
    for value in range(len(empty_1)):
        empty_3.append(value)
        
    for x,y,i,j in zip(empty_1, empty_2, empty, empty_3):
        chisquare = chi_square(X=x, Y=y, Z=[], data=df, significance_level=0.05, boolean=False) #returns chi, p_value, dof
        empty_4.append([j, i , chisquare])
        
    return empty_4


### log-likelihood tests

In [6]:
def g2val(df):
#returns a tuple(chi2, p_value, dof) if boolean = false
#the null hypothesis is that they are independent of each other
#if true, the p_value is higher than the significance test, we do not reject the null hypothesis
#if false, the p_value is lower than the significance test, we reject the null hypothesis
    v = list(df)
    empty = []
    empty_1 = []
    empty_2 = []
    empty_3 = []
    empty_4 = []
    itertools_combinations = list(itertools.combinations(v, 2)) #finds every possible combination of list(df)
    for i in itertools_combinations:
        empty_1.append(i[0])
        empty_2.append(i[1])
        
    total_number_of_combinations = len(empty_1)
    
    for t in range(len(itertools_combinations)):
        empty.append(itertools_combinations[t])
        
    for value in range(len(empty_1)):
        empty_3.append(value)
        
    for x,y,i,j in zip(empty_1, empty_2, empty, empty_3):
        g2 = log_likelihood(X=x, Y=y, Z=[], significance_level=0.05, boolean=True, data=df) 
        empty_4.append([j, i , g2])
        
    return empty_4

### Sorting Functions

In [7]:
def sortReturn(data):
    #just a sorting function
    true_list = []
    false_list = []
    for i in data:
        if True in i:
            true_list.append(i)
        if False in i:
            false_list.append(i)
    return true_list, false_list

In [8]:
#returns a list with a tuple of combinations of false
def false2tuple(data):
    empty = []
    false_list_of_tuples = []
    new_df = pd.DataFrame(data)
    newer_df = new_df[1]
    return newer_df

In [9]:
#given 2 lists, append them together and remove duplicates
def remove_dupes(list_1, list_2):
    for i in list_2:
        list_1.append(i)
    remove_dupes = list(set(list_1))
    return remove_dupes

### CONDITIONAL MAIN CODE

In [10]:
#returns TRUE/FALSE Xs and Ys in a tuple of (X,Y)
def conditional_sort(df):
    output_false = []
    output_true = []
    my_list = [x for x in chi2bool(df) if False in x]
    my_list_2 = [x for x in chi2bool(df) if True in x]
    for i,j in zip(my_list, my_list_2):
        output_true.append(j[1])
        output_false.append(i[1])
    return output_true, output_false

In [11]:
#returns permutations of x,y,z where x,y does not repeat e.g (a,b,c), (b,a,c)
def conditional_permute(df):
    a = conditional_sort(df)[1] #obtains the list of false outputs, false output means dependent
    k = list(itertools.permutations(df,3))
    permute_list = []
    for i in a:
        permute = list(itertools.permutations(i))
        permute_list.append(permute[1]) #returns a list of permuted items from a
    for items in permute_list:
        a.append(items) #adds all possible permutations to a 
    my_list = [x for x in k if all(x[:2] != y[:2] for y in a)] #checks if all elements in x[:2] != y[:2]
    my_list_2 = [x[:2] for x in my_list]
    a0 = list(tuple(sorted(l)) for l in my_list_2)
    output = [x for x in my_list if x[:2] in a0] #the three lines fixes all the permutations 
    return output

In [12]:
def conditional_1st_order(df):
    b0 = conditional_permute(df)
    b_100 = []
    for i,j,k in b0:
        chi2 = chi_square(X=i, Y=j, Z=[k], data=df, significance_level=0.05)
        b_100.append((i,j,k, chi2))
    b_100_true = [x for x in b_100 if True in x]
    b_100_false = [x for x in b_100 if False in x]
    return b_100_true, b_100_false

In [14]:
# def PC_algorithm(n, df):
    

In [15]:
cond_0 = chi2bool(df_test)
true_list_0 = [x[1] for x in cond_0 if True in x]

In [16]:
%%time
cond_1 = conditional_1st_order(df_test)

CPU times: total: 2min 30s
Wall time: 2min 32s


In [18]:
true_list_1 = [x[:2] for x in cond_1[0]]
true_list = true_list_0 + true_list_1
len(true_list), len(true_list_0), len(true_list_1)

(13286, 401, 12885)

In [19]:
chi2_ind = list(set(true_list))
len(chi2_ind)

648

In [22]:
v = list(df_test)
k = list(itertools.permutations(v, 2))
t = [x for x in k if x[:2] not in chi2_ind]
t

[('PULMEMBOLUS', 'PAP'),
 ('PULMEMBOLUS', 'SHUNT'),
 ('PULMEMBOLUS', 'SAO2'),
 ('PAP', 'PULMEMBOLUS'),
 ('PAP', 'SHUNT'),
 ('KINKEDTUBE', 'VENTLUNG'),
 ('KINKEDTUBE', 'VENTALV'),
 ('KINKEDTUBE', 'PVSAT'),
 ('KINKEDTUBE', 'SAO2'),
 ('KINKEDTUBE', 'ARTCO2'),
 ('KINKEDTUBE', 'EXPCO2'),
 ('KINKEDTUBE', 'PRESS'),
 ('KINKEDTUBE', 'MINVOL'),
 ('INTUBATION', 'VENTLUNG'),
 ('INTUBATION', 'SHUNT'),
 ('INTUBATION', 'VENTALV'),
 ('INTUBATION', 'PVSAT'),
 ('INTUBATION', 'SAO2'),
 ('INTUBATION', 'ARTCO2'),
 ('INTUBATION', 'CATECHOL'),
 ('INTUBATION', 'HR'),
 ('INTUBATION', 'HREKG'),
 ('INTUBATION', 'EXPCO2'),
 ('INTUBATION', 'PRESS'),
 ('INTUBATION', 'MINVOL'),
 ('INTUBATION', 'HRSAT'),
 ('MINVOLSET', 'KINKEDTUBE'),
 ('MINVOLSET', 'INTUBATION'),
 ('MINVOLSET', 'VENTMACH'),
 ('MINVOLSET', 'VENTTUBE'),
 ('MINVOLSET', 'VENTLUNG'),
 ('MINVOLSET', 'VENTALV'),
 ('MINVOLSET', 'PVSAT'),
 ('MINVOLSET', 'SAO2'),
 ('MINVOLSET', 'ARTCO2'),
 ('MINVOLSET', 'CATECHOL'),
 ('MINVOLSET', 'HR'),
 ('MINVOLSET', 'HREKG'

In [25]:
my_list = [x for x in itertools.permutations(v, N) if x[:2] not in chi2_ind]
my_list
m_1 = [x[0] for x in my_list]
m_2 = [x[1] for x in my_list]
m_3 = [x[2:] for x in my_list]
m_1
for i,j,k in zip(m_1, m_2, m_3):
    p = chi_square(X=i, Y=j, Z=k, data=df_test, significance_level=0.05)
    print(k)

NameError: name 'N' is not defined

In [32]:
%%time
N = 2
testing_list = []
my_list = [x for x in itertools.permutations(v, N) if x[:2] not in chi2_ind]
m_1 = [x[0] for x in my_list]
m_2 = [x[1] for x in my_list]
m_3 = [x[2:] for x in my_list]
#         print(i[0], i[1], i[2:N])
for i,j,k in zip(m_1, m_2, m_3):
    phi = chi_square(X=i, Y=j, Z=k, data=df_test, significance_level=0.05)
    testing_list.append([i,j,k,phi])
testing_list

CPU times: total: 1.27 s
Wall time: 1.27 s


[['PULMEMBOLUS', 'PAP', (), False],
 ['PULMEMBOLUS', 'SHUNT', (), False],
 ['PULMEMBOLUS', 'SAO2', (), False],
 ['PAP', 'PULMEMBOLUS', (), False],
 ['PAP', 'SHUNT', (), False],
 ['KINKEDTUBE', 'VENTLUNG', (), False],
 ['KINKEDTUBE', 'VENTALV', (), False],
 ['KINKEDTUBE', 'PVSAT', (), False],
 ['KINKEDTUBE', 'SAO2', (), False],
 ['KINKEDTUBE', 'ARTCO2', (), False],
 ['KINKEDTUBE', 'EXPCO2', (), False],
 ['KINKEDTUBE', 'PRESS', (), False],
 ['KINKEDTUBE', 'MINVOL', (), False],
 ['INTUBATION', 'VENTLUNG', (), False],
 ['INTUBATION', 'SHUNT', (), False],
 ['INTUBATION', 'VENTALV', (), False],
 ['INTUBATION', 'PVSAT', (), False],
 ['INTUBATION', 'SAO2', (), False],
 ['INTUBATION', 'ARTCO2', (), False],
 ['INTUBATION', 'CATECHOL', (), False],
 ['INTUBATION', 'HR', (), False],
 ['INTUBATION', 'HREKG', (), False],
 ['INTUBATION', 'EXPCO2', (), False],
 ['INTUBATION', 'PRESS', (), False],
 ['INTUBATION', 'MINVOL', (), False],
 ['INTUBATION', 'HRSAT', (), False],
 ['MINVOLSET', 'KINKEDTUBE', (),

In [33]:
true_test = [x for x in testing_list if True in x]
true_test, len(true_test) #154

([['MINVOLSET', 'KINKEDTUBE', (), True],
  ['MINVOLSET', 'INTUBATION', (), True],
  ['VENTMACH', 'PULMEMBOLUS', (), True],
  ['VENTMACH', 'PAP', (), True],
  ['VENTMACH', 'KINKEDTUBE', (), True],
  ['VENTMACH', 'INTUBATION', (), True],
  ['VENTTUBE', 'PULMEMBOLUS', (), True],
  ['VENTTUBE', 'PAP', (), True],
  ['VENTTUBE', 'KINKEDTUBE', (), True],
  ['VENTTUBE', 'INTUBATION', (), True],
  ['VENTLUNG', 'PULMEMBOLUS', (), True],
  ['VENTLUNG', 'PAP', (), True],
  ['SHUNT', 'KINKEDTUBE', (), True],
  ['SHUNT', 'MINVOLSET', (), True],
  ['SHUNT', 'DISCONNECT', (), True],
  ['VENTALV', 'PULMEMBOLUS', (), True],
  ['VENTALV', 'PAP', (), True],
  ['FIO2', 'DISCONNECT', (), True],
  ['PVSAT', 'PULMEMBOLUS', (), True],
  ['PVSAT', 'PAP', (), True],
  ['SAO2', 'PAP', (), True],
  ['INSUFFANESTH', 'DISCONNECT', (), True],
  ['INSUFFANESTH', 'FIO2', (), True],
  ['TPR', 'PULMEMBOLUS', (), True],
  ['TPR', 'PAP', (), True],
  ['TPR', 'KINKEDTUBE', (), True],
  ['TPR', 'INTUBATION', (), True],
  ['T

In [57]:
m3 = [x[2] for x in my_list]
m3

['KINKEDTUBE',
 'INTUBATION',
 'MINVOLSET',
 'VENTMACH',
 'DISCONNECT',
 'VENTTUBE',
 'VENTLUNG',
 'SHUNT',
 'VENTALV',
 'FIO2',
 'PVSAT',
 'SAO2',
 'INSUFFANESTH',
 'ARTCO2',
 'ANAPHYLAXIS',
 'TPR',
 'CATECHOL',
 'HR',
 'ERRCAUTER',
 'HREKG',
 'HYPOVOLEMIA',
 'LVFAILURE',
 'LVEDVOLUME',
 'CVP',
 'STROKEVOLUME',
 'CO',
 'BP',
 'EXPCO2',
 'ERRLOWOUTPUT',
 'PRESS',
 'HRBP',
 'MINVOL',
 'HISTORY',
 'HRSAT',
 'PCWP',
 'PAP',
 'KINKEDTUBE',
 'INTUBATION',
 'MINVOLSET',
 'VENTMACH',
 'DISCONNECT',
 'VENTTUBE',
 'VENTLUNG',
 'VENTALV',
 'FIO2',
 'PVSAT',
 'SAO2',
 'INSUFFANESTH',
 'ARTCO2',
 'ANAPHYLAXIS',
 'TPR',
 'CATECHOL',
 'HR',
 'ERRCAUTER',
 'HREKG',
 'HYPOVOLEMIA',
 'LVFAILURE',
 'LVEDVOLUME',
 'CVP',
 'STROKEVOLUME',
 'CO',
 'BP',
 'EXPCO2',
 'ERRLOWOUTPUT',
 'PRESS',
 'HRBP',
 'MINVOL',
 'HISTORY',
 'HRSAT',
 'PCWP',
 'PAP',
 'KINKEDTUBE',
 'INTUBATION',
 'MINVOLSET',
 'VENTMACH',
 'DISCONNECT',
 'VENTTUBE',
 'VENTLUNG',
 'SHUNT',
 'VENTALV',
 'FIO2',
 'PVSAT',
 'INSUFFANESTH',
 'AR

In [55]:
testing_list

[['PULMEMBOLUS', 'PAP', ('KINKEDTUBE',), False],
 ['PULMEMBOLUS', 'PAP', ('INTUBATION',), False],
 ['PULMEMBOLUS', 'PAP', ('MINVOLSET',), False],
 ['PULMEMBOLUS', 'PAP', ('VENTMACH',), False],
 ['PULMEMBOLUS', 'PAP', ('DISCONNECT',), False],
 ['PULMEMBOLUS', 'PAP', ('VENTTUBE',), False],
 ['PULMEMBOLUS', 'PAP', ('VENTLUNG',), False],
 ['PULMEMBOLUS', 'PAP', ('SHUNT',), False],
 ['PULMEMBOLUS', 'PAP', ('VENTALV',), False],
 ['PULMEMBOLUS', 'PAP', ('FIO2',), False],
 ['PULMEMBOLUS', 'PAP', ('PVSAT',), False],
 ['PULMEMBOLUS', 'PAP', ('SAO2',), False],
 ['PULMEMBOLUS', 'PAP', ('INSUFFANESTH',), False],
 ['PULMEMBOLUS', 'PAP', ('ARTCO2',), False],
 ['PULMEMBOLUS', 'PAP', ('ANAPHYLAXIS',), False],
 ['PULMEMBOLUS', 'PAP', ('TPR',), False],
 ['PULMEMBOLUS', 'PAP', ('CATECHOL',), False],
 ['PULMEMBOLUS', 'PAP', ('HR',), False],
 ['PULMEMBOLUS', 'PAP', ('ERRCAUTER',), False],
 ['PULMEMBOLUS', 'PAP', ('HREKG',), False],
 ['PULMEMBOLUS', 'PAP', ('HYPOVOLEMIA',), False],
 ['PULMEMBOLUS', 'PAP', ('

In [53]:
%%time
N = 2
n = 2
v = list(df_test)
while N <= n:
    my_list = [x for x in itertools.permutations(v, N) if x[:2] not in chi2_ind]
    m_1 = [x[0] for x in my_list]
    m_2 = [x[1] for x in my_list]
    m_3 = [x[2:] for x in my_list]
    #         print(i[0], i[1], i[2:N])
    for i,j,k in zip(m_1, m_2, m_3):
        phi = chi_square(X=i, Y=j, Z=k, data=df_test, significance_level=0.05)
        
#         print(chi2)
    N += 1

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
True
True
False
False
False
False
False
False
False
False
False
False
False
True
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fal

### Dead functions which i may or may not remove

In [ ]:
#returns combinations of Zs
def conditional_combine_1(df):
    first_empty = []
    second_empty = []
    my_list = conditional_sort_2(df)
    for i in my_list:
        first_empty.append(i[0])
        second_empty.append(i[1])
    list_df = list(df)
    v = list(itertools.combinations(list_df, 2))
    for i,j,k in zip(first_empty, second_empty, v):
        chi_square(X=i, Y=j, Z=k, data=df)
    return chi_square